In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imgaug as ia
import torch
import pandas as pd
import os
import glob
import os
import warnings

In [2]:
def bb_midpoint_to_corner(bb):
    label = bb[0]
    x1 = bb[1] - bb[3]/2
    x2 = bb[1] + bb[3]/2
    y1 = bb[2] - bb[4]/2
    y2 = bb[2] + bb[4]/2
    # A: area will only be used for sorting
    area = bb[3]*bb[4]
    corner_list = [label, x1, x2, y1, y2, area]
    return np.array(corner_list)

def open_yolo_sort(path, image_name):
    try:
        image = cv2.imread(path + image_name)
        shape = image.shape
        width = shape[1]
        height = shape[0]
        label = path + os.path.splitext(image_name)[0] + ".txt"
        boxes = np.genfromtxt(label, delimiter=' ')
        bb = boxes
        # reshaping the np array is necessary in case a file with a single box is read
        boxes = boxes.reshape(boxes.size//5, 5)
        #print(boxes.shape)
        boxes = np.apply_along_axis(bb_midpoint_to_corner, axis=1, arr=boxes)
        # A: sorting by area
        boxes = boxes[boxes[:, 5].argsort()]
        # A: reversing the sorted list so bigger areas come first
        boxes = boxes[::-1]
        return image, boxes, width, height
    except Exception as e:
        #print(e)
        print(image_name)
        return image, None, None, None

def create_segclass(image_path, save_path, image_name):
    image, bb, w, h = open_yolo_sort(image_path, image_name)
    image_copy = image.copy()*0 
    if bb is not None:       
        for label, x1, x2, y1, y2, area in bb:
            # A: the white outline with four pixels of thickness
            cv2.rectangle(image_copy, (int(x1*w), int(y1*h)), (int(x2*w), int(y2*h)), (255, 255, 255), 4)
            # A: the class coded filing, specified by -1
            cv2.rectangle(image_copy, (int(x1*w), int(y1*h)), (int(x2*w), int(y2*h)), colors[int(label)], -1)
        image_copy = cv2.cvtColor(image_copy, cv2.COLOR_BGR2RGB)
        cv2.imwrite(save_path + os.path.splitext(image_name)[0] + ".png", image_copy)
    else:
        cv2.imwrite(save_path + os.path.splitext(image_name)[0] +".png", image_copy)

def create_segclass_filtered(image_path, save_path, image_name, filter_list):
    image, bb, w, h = open_yolo_sort(image_path, image_name)
    image_copy = image.copy()*0 
    if bb is not None:       
        for label, x1, x2, y1, y2, area in bb:
            if label not in filter_list:
                # A: the white outline with four pixels of thickness
                cv2.rectangle(image_copy, (int(x1*w), int(y1*h)), (int(x2*w), int(y2*h)), (255, 255, 255), 4)
                # A: the class coded filing, specified by -1
                cv2.rectangle(image_copy, (int(x1*w), int(y1*h)), (int(x2*w), int(y2*h)), colors[int(label)], -1)
        image_copy = cv2.cvtColor(image_copy, cv2.COLOR_BGR2RGB)
        cv2.imwrite(save_path + os.path.splitext(image_name)[0] + ".png", image_copy)
    else:
        cv2.imwrite(save_path + os.path.splitext(image_name)[0] +".png", image_copy)
        
def create_boxi_filtered(image_path, save_path, image_name, filter_list):
    image, bb, w, h = open_yolo_sort(image_path, image_name)
    image_copy = image.copy()*0 
    if bb is not None:       
        for label, x1, x2, y1, y2, area in bb:
            if label in filter_list:
                rw = (x2*w - x1*w)
                rh = (y2*h - y1*h)
                # the white box that will be ignored
                cv2.rectangle(image_copy, (int(x1*w), int(y1*h)), (int(x2*w), int(y2*h)), (255, 255, 255), -1)
                # the colored class filling that amounts to 20% of the inner region
                cv2.rectangle(image_copy, (int(x1*w + rw*0.4), int(y1*h + rh*0.4)), (int(x2*w - rw*0.4), int(y2*h - rh*0.4)), colors[int(label)], -1)
        image_copy = cv2.cvtColor(image_copy, cv2.COLOR_BGR2RGB)
        cv2.imwrite(save_path + os.path.splitext(image_name)[0] + ".png", image_copy)
    else:
        cv2.imwrite(save_path + os.path.splitext(image_name)[0] +".png", image_copy)

colors = [(162, 0, 255), # Chave seccionadora lamina (Aberta)
         (97, 16, 162),   # Chave seccionadora lamina (Fechada)
         (81, 162, 0),    # Chave seccionadora tandem (Aberta)
         (48, 97, 165),   # Chave seccionadora tandem (Fechada)
         (121, 121, 121), # Disjuntor
         (255, 97, 178),  # Fusivel
         (154, 32, 121),  # Isolador disco de vidro
         (255, 255, 125), # Isolador pino de porcelana
         (162, 243, 162), # Mufla
         (143, 211, 255), # Para-raio
         (40, 0, 186),    # Religador
         (255, 182, 0),   # Transformador
         (138, 138, 0),   # Transformador de Corrente (TC)
         (162, 48, 0),    # Transformador de Potencial (TP)
         (162, 0, 96)     # Chave tripolar
         ] 

In [7]:
# paths = ["/home/jovyan/work/yolo_og/yolov3/Data/15_classes/train/",            # 2363
#         "/home/jovyan/work/yolo_og/yolov3/Data/15_classes/flir_train/",       # 1083
#         "/home/jovyan/work/yolo_og/yolov3/Data/15_classes/night_train/",      # 479
#         "/home/jovyan/work/yolo_og/yolov3/Data/15_classes/tripolares_train/", # 879
#         "/home/jovyan/work/yolo_og/yolov3/Data/15_classes/fake_night/"]       # 311

paths = ["../data/train/"]

# filter list está no modo in
filter_list = [10] # religador
save_path = "../data/religador/train/"

check_path = os.path.isdir(save_path)
if not check_path:
    os.makedirs(save_path)
    print("created folder: ", save_path) 
#label_path = "../data/labels_17class/"
for image_path in paths:
    file_list = os.path.join(os.path.join(image_path, "*.*"))
    image_list = []
    file_list = glob.glob(file_list)
    for name in file_list:
        if "txt" not in name:
            image_list.append(name.split("/")[-1])

    for file in image_list:
#         create_segclass_filtered(image_path, save_path, file, filter_list)
        create_boxi_filtered(image_path, save_path, file, filter_list)